# Image blending using openCV
* Image Blending techniques mainly focus on two major aspects for creating a blended image by using Image Pyramids, namely Gaussian and Laplacian Pyramids. A pyramid or pyramid representation is a type of multi-scale representation where an image is repeatedly smoothened and subsampled. It searches for a target pattern over many repetitive scales. At first from the image, it creates copies of the target pattern and reconstructs it over several expanded scales. Each copy is then convolved with the original image, creating a new and unique image.

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

Printing The Shape and Size of Our Images Imported


In [2]:
img1 = cv.imread('apple.jpg') 
img2 = cv.imread('orange.jpg')
print(img1.shape)
print(img2.shape)

Dividing them into right and left half as tuples using hstack 


In [4]:
apple_orange = np.hstack((img1[:,:256],img2[:,256:]))

In [6]:
# generate Gaussian pyramid for apple
apple_copy = img1.copy()
gp_apple = [apple_copy]
for i in range(6):
    apple_copy = cv.pyrDown(apple_copy)
    gp_apple.append(apple_copy)

# generate gaussian pyramid for orange
orange_copy = img2.copy()
gp_orange = [orange_copy]
for i in range(6):
    orange_copy = cv.pyrDown(orange_copy)
    gp_orange.append(orange_copy)

# generate Laplacian pyramid for apple
apple_copy = gp_apple[5]
lp_apple = [apple_copy]
for i in range(5, 0 , -1):
    gaussian_expanded = cv.pyrUp(gp_apple[i]) 
    laplacian = cv.subtract(gp_apple[i-1],gaussian_expanded)
    lp_apple.append(laplacian)

# generate laplacian pyramid for orange
orange_copy = gp_orange[5]
lp_orange = [orange_copy]
for i in range(5, 0, -1):
    gaussian_expanded = cv.pyrUp(gp_orange[i])
    laplacian = cv.subtract(gp_orange[i-1],gaussian_expanded)
    lp_orange.append(laplacian)
    
# now add left and right halves of images in each level
apple_orange_pyramid = []
n = 0
for apple_lap , orange_lap in zip(lp_apple,lp_orange):
    n += 1
    cols, rows , ch = apple_lap.shape
    laplacian = np.hstack((apple_lap[:, :int(cols/2)], orange_lap[:,int(cols/2):]))
    apple_orange_pyramid.append(laplacian)

# now reconstruct
apple_orange_reconstruct = apple_orange_pyramid[0]
for i in range(1,6):
    apple_orange_reconstruct = cv.pyrUp(apple_orange_reconstruct)
    apple_orange_reconstruct = cv.add(apple_orange_pyramid[i],apple_orange_reconstruct)


In [7]:
cv.imshow('orange',img1)
cv.imshow('apple',img2)
cv.imshow('apple_orange_reconstruct',apple_orange_reconstruct)


cv.waitKey(0)
cv.destroyAllWindows()